In [1]:
import pandas as pd
import os
import regex as re
import string
import numpy as np
import collections

import contractions
import spacy
from spacy.tokenizer import Tokenizer
import en_core_web_sm
import nltk
from nltk.stem import WordNetLemmatizer
#import sys



In [ ]:
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

In [2]:


#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = en_core_web_sm.load()

# #alter default spacy stopwords so 'not' isn't considered a stop word
customize_stop_words = [
    'not'
]
for w in customize_stop_words:
    nlp.vocab[w].is_stop = False
    

# #Should hopefully deal with contractions
# special_cases = {":)": [{"ORTH": ":)"}]}


    
    
# def custom_tokenizer(nlp, custom_stop_words):
    
#     for w in customize_stop_words:
#         nlp.vocab[w].is_stop = False
        
#     return Tokenizer(nlp.vocab, rules=special_cases)

# nlp.tokenizer = custom_tokenizer(nlp, customize_stop_words)

In [ ]:
#https://www.kaggle.com/balatmak/text-preprocessing-steps-and-universal-pipeline



In [3]:
#sys.path.append("../data/raw") 
os.chdir("/Users/amywinecoff/Documents/CITP/Research/Github/ai-dialogues/startup-study/data/raw")

In [4]:
#replace some words so all texts refer to them the same way throughout
lookp_dict = {
    "artificial intelligence" : "AI", 
    "machine learning" : "ML",
    }

In [5]:
transcript = open('P1.txt', "r", encoding='utf-8')

#This is sloppy, but I couldn't figure out how to get this to work with regex
transcript = transcript.read()
transcript = " ".join(lookp_dict.get(ele, ele) for ele in transcript.split()).split('Speaker ')
transcript = ''.join([e.replace('\n', '').replace("\'", "") for e in transcript if len(e)> 0 and e.startswith("2:")])
transcript = contractions.fix(re.sub('2:', ' ', transcript))
print(transcript)

  Sure, so on GrowthChannel we are a software technology company based in Austin. We have been around for about a year from now since we launched publicly. We automated marketing planning. We work with startups, SMBs, agencies and enterprise marketing teams. So usually marketers would use our solution to automate market research, identify their ideal customer profiles and help their customer decision journey. Our tool is also AI based, which we use ourselves, of course. We also use a few other tools ourselves, which are also AI based.   We help you save time and also identify growth opportunities for your startups, SMB, or enterprise marketing team. Agencies also use our solution to help them scale faster. So let us say right now they considering the current resources, they might be doing one marketing plan with their resources and our tool can multiply it by 20.   I am the founder.   That was a lot of things.   Yeah. All right, so we do small promotion and customer feedback about one 

In [7]:
# doc = nlp(transcript)
# # #spacy_tokens = [token.text for token in doc]
# # #print(spacy_tokens)



# punc_removed = " ".join([t.text for t in doc if t.text not in string.punctuation])

# spacy_tokens = [t.lemma_ for t in nlp(punc_removed) if not t.is_stop]

spacy_lemmas = [t.lemma_ for t in nlp(transcript) if t.text not in string.punctuation and not t.is_stop]

spacy_text = " ".join(spacy_lemmas)
#text_without_stop_words = [t.text for t in nlp(text) if not t.is_stop]
spacy_text

'   sure GrowthChannel software technology company base Austin year launch publicly automate marketing planning work startup smb agency enterprise marketing team usually marketer use solution automate market research identify ideal customer profile help customer decision journey tool AI base use course use tool AI base    help save time identify growth opportunity startup SMB enterprise marketing team agency use solution help scale fast let right consider current resource marketing plan resource tool multiply 20    founder    lot thing    yeah right small promotion customer feedback feature update platform publish share information startup interview prior Clubhouse check social schedule check analytic weekly analytic check team plan week talk customer demo check upcoming event preparation kind thing let yeah think main thing    yeah month co host startup interview usually startup story early stage startup like Indie Hacker Maker small team early stage not fund growth traction talk grow

In [ ]:
# #Note that this current form does remove "not" as a stop word, which may be an issue later on

# tokens = [t for t in nltk.word_tokenize(text) if t not in string.punctuation and t not in nltk.corpus.stopwords.words('english')]
# #print(len(tokens))

In [ ]:
# wordnet_lemmatizer = WordNetLemmatizer()
# lemmatize_words = np.vectorize(wordnet_lemmatizer.lemmatize)
# lemmatized_text = ' '.join(lemmatize_words(tokens))

In [ ]:
# print(lemmatized_text)
    

In [ ]:
counts_no_urls = collections.Counter(all_words_no_urls)